In [1]:
import numpy as np
from serpentGenerator.functions.pin import pin
from serpentGenerator.data.materialLibrary import MATLIB
from serpentGenerator.functions.material import material
from serpentGenerator.functions.sqLattice import sqLat
from serpentGenerator.functions.hexLattice import hexLat
from serpentGenerator.functions.pinStack import pinStack
from serpentGenerator.functions.surf import surf
from serpentGenerator.functions.lats import lats
from serpentGenerator.functions.mats import mats
from serpentGenerator.functions.pins import pins
from serpentGenerator.functions.core import core
from serpentGenerator.functions.housing import housing

# XS Generation for a 3D Small Modular PWR
1. Homogenized few group parameters will be generated for a standard 3D PWR assembly
2. The assembly will experience no burnup
3. The assembly will experience no perturbations
4. The assembly will have predefined values for the axial fuel temperatures and coolant densities
taken from a thermo-hydraulic solver
5. The assembly has reflector on the top and bottom axial layers
6. The XS generation will be done using a 2 group structure where the XS will be generated for the fuel and reflector universes

## Assembly Geometry
1. The Assembly will consist of a 19 x 19 sqaure lattice
2. Each channel will have 22 axial layers, with the 1st and 22nd layer being refectors.
3. Each axial layer will have its own material and conditions pre determined by a thermal hyrdaulic solver.


In [2]:
nAxLayers = 22 #number of axial layers #20 Active Layers 2 Reflector Layers

nFA1 = 9
nFA2 = 12
nFA3 = 16

#Pre generated fuel temps and coolant densities
fuelTempsFA1 = np.zeros((nFA1, 20)) + 900
fuelTempsFA2 = np.zeros((nFA2, 20)) + 900
fuelTempsFA3 = np.zeros((nFA3, 20)) + 900
coolDensFA1 = np.zeros((nFA1, 20)) + 600
coolDensFA2 = np.zeros((nFA2, 20)) + 600
coolDensFA3 = np.zeros((nFA3, 20)) + 600

## Generating Materials
1. Since 2 of the 22 axial layers will be reflectors, this means we will have 20 active layers. 
2. The thermohydraulic solver has pretedetermined the temperatures of the 20 active layers for the fuel and coolant densities.
3. For the fuel we will be using UO2 and lightWater for the coolant. 

In [3]:
fuel1 = MATLIB['UO2'].duplicateMat("fuelFA1")
fuel1.set('fractions', np.array([-0.04319337211, -0.83830401789, -.11850261])) # 4.9 % wt enrichment
fuel1.set('dens', -10.5216)
fuel1.set('rgb', "238 59 49")

fuel2 = fuel1.duplicateMat("fuelFA2")
fuel2.set('rgb', "255 165 0")

fuel3 = fuel1.duplicateMat("fuelFA3")
fuel3.set('rgb', "255 255 0")

ublank1 = fuel1.duplicateMat("upperfuelBlanketFA1")
ublank1.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
ublank1.set('rgb', "255 255 255")

ublank2 = fuel1.duplicateMat("upperfuelBlanketFA2")
ublank2.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
ublank2.set('rgb', "255 255 255")

ublank3 = fuel1.duplicateMat("upperfuelBlanketFA3")
ublank3.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
ublank3.set('rgb', "255 255 255")

lblank1 = fuel1.duplicateMat("lowerfuelBlanketFA1")
lblank1.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
lblank1.set('rgb', "255 255 255")

lblank2 = fuel1.duplicateMat("lowerBlanketFA2")
lblank2.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
lblank2.set('rgb', "255 255 255")

lblank3 = fuel1.duplicateMat("lowerBlanketFA3")
lblank3.set('fractions', np.array([-0.017629948, -0.863867442, -.11850261]))# 2.0 % wt enrichment 
lblank3.set('rgb', "255 255 255")


mod1 = MATLIB['H2O'].duplicateMat("modFA1")
mod1.set('rgb', "198 226 255")
mod2 = MATLIB['H2O'].duplicateMat("modFA2")
mod2.set('rgb', "198 226 255")
mod3 = MATLIB['H2O'].duplicateMat("modFA3")
mod3.set('rgb', "198 226 255")

clad = MATLIB['Zr'].duplicateMat("clad")
clad.set('rgb', "183 183 183")

ref = MATLIB['Reflector'].duplicateMat("reflector") # reflector plenum
ref.set('rgb', "204 153 255")

ba = MATLIB['B'].duplicateMat("B4C") # 90% wt enriched b-10

MB = 10.8
MC = 12
MB4C = MB*4 + MC
rhoB4C = 2.52
AvroNorm = .6022
nb4c = rhoB4C*AvroNorm/MB4C
nc12 = nb4c
b10_enrich = .90
b11_enrich = 1 - b10_enrich
nb10 = nb4c*b10_enrich
nb11 = nb4c*b11_enrich

b4cdens = -2.52
ba.set('rgb', "255 255 0")
ba.set('nuclides', np.array([5010, 5011, 6012]))
ba.set('fractions', np.array([nb10, nb11, nc12]))
ba.set('dens', -1*rhoB4C)
ba.set('isBurn', True)

In [4]:
RT = pin("RT", 1) #top reflector
RT.set('materials', [ref])

RB = pin("RB", 1) #bottom reflector
RB.set('materials', [ref])

In [5]:
fuelMatsFA1 = np.zeros(fuelTempsFA1.shape).tolist()
coolMatsFA1 = np.zeros(fuelTempsFA1.shape).tolist()
blankMatsFA1 = np.zeros(fuelTempsFA1.shape).tolist()
fuelMatsFA2 = np.zeros(fuelTempsFA2.shape).tolist()
coolMatsFA2 = np.zeros(fuelTempsFA2.shape).tolist()
fuelMatsFA3 = np.zeros(fuelTempsFA3.shape).tolist()
coolMatsFA3 = np.zeros(fuelTempsFA3.shape).tolist()

for i in range(0, fuelTempsFA1.shape[0]):
    for j in range(0, fuelTempsFA1.shape[1]):
        if j == 0:
            fuelMatsFA1[i][j] = lblank1.duplicateMat(lblank1.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            fuelMatsFA1[i][j].set('temp', fuelTempsFA1[i][j])
            coolMatsFA1[i][j] = mod1.duplicateMat(mod1.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            coolMatsFA1[i][j].set('dens', coolDensFA1[i][j])
        elif (j == (fuelTempsFA1.shape[1] - 1)):
            fuelMatsFA1[i][j] =  ublank1.duplicateMat(ublank1.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            fuelMatsFA1[i][j].set('temp', fuelTempsFA1[i][j])
            coolMatsFA1[i][j] = mod1.duplicateMat(mod1.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            coolMatsFA1[i][j].set('dens', coolDensFA1[i][j])
        else:
            fuelMatsFA1[i][j] = fuel1.duplicateMat(fuel1.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            fuelMatsFA1[i][j].set('temp', fuelTempsFA1[i][j])
            coolMatsFA1[i][j] = mod1.duplicateMat(mod1.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            coolMatsFA1[i][j].set('dens', coolDensFA1[i][j])

for i in range(0, fuelTempsFA2.shape[0]):
    for j in range(0, fuelTempsFA2.shape[1]):
        if j == 0:
            fuelMatsFA2[i][j] =  lblank2.duplicateMat(lblank2.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            fuelMatsFA2[i][j].set('temp', fuelTempsFA2[i][j])
            coolMatsFA2[i][j] = mod2.duplicateMat(mod2.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            coolMatsFA2[i][j].set('dens', coolDensFA2[i][j])
        elif (j == (fuelTempsFA2.shape[1] - 1)):
            fuelMatsFA2[i][j] =  ublank2.duplicateMat(ublank2.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            fuelMatsFA2[i][j].set('temp', fuelTempsFA2[i][j])
            coolMatsFA2[i][j] = mod1.duplicateMat(mod2.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            coolMatsFA2[i][j].set('dens', coolDensFA2[i][j])
        else:
            fuelMatsFA2[i][j] = fuel2.duplicateMat(fuel2.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            fuelMatsFA2[i][j].set('temp', fuelTempsFA2[i][j])
            coolMatsFA2[i][j] = mod2.duplicateMat(mod2.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            coolMatsFA2[i][j].set('dens', coolDensFA2[i][j])
        
for i in range(0, fuelTempsFA3.shape[0]):
    for j in range(0, fuelTempsFA3.shape[1]):
        if j == 0:
            fuelMatsFA3[i][j] = lblank3.duplicateMat(lblank3.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            fuelMatsFA3[i][j].set('temp', fuelTempsFA3[i][j])
            coolMatsFA3[i][j] = mod3.duplicateMat(mod3.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            coolMatsFA3[i][j].set('dens', coolDensFA3[i][j])
        elif (j == (fuelTempsFA3.shape[1] - 1)):
            fuelMatsFA3[i][j] =  ublank3.duplicateMat(ublank3.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            fuelMatsFA3[i][j].set('temp', fuelTempsFA3[i][j])
            coolMatsFA3[i][j] = mod3.duplicateMat(mod3.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            coolMatsFA3[i][j].set('dens', coolDensFA3[i][j])
        else:
            fuelMatsFA3[i][j] = fuel3.duplicateMat(fuel3.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            fuelMatsFA3[i][j].set('temp', fuelTempsFA3[i][j])
            coolMatsFA3[i][j] = mod3.duplicateMat(mod3.id + "_Assem"+str(i+1) + "Z"+str(j+1))
            coolMatsFA3[i][j].set('dens', coolDensFA3[i][j])


## Generating Individual Channel Axial Layers
1. To model an individual channel axial layer we will be using the serpent pin geometry
2. Each indivial pin will have its own set of materials
3. There will be 4 types of pin models, fuel, guide, gap, and reflector pins


In [6]:
ro_fuel = .4096
ro_clad = .4760
ri_gt = .5531
ro_gt = .6133

fuelPinRadii = [ro_fuel, ro_clad]
guideTubeRadii = [ri_gt, ro_gt]

fuelPinsFA1 = np.zeros(fuelTempsFA1.shape).tolist()
# blankPinsFA1 = np.zeros(fuelTempsFA1.shape).tolist()
guidePinsFA1 = np.zeros(fuelTempsFA1.shape).tolist()
gapPinsFA1 = np.zeros(fuelTempsFA1.shape).tolist()

fuelPinsFA2 = np.zeros(fuelTempsFA2.shape).tolist()
# blankPinsFA2 = np.zeros(fuelTempsFA2.shape).tolist()
guidePinsFA2 = np.zeros(fuelTempsFA2.shape).tolist()
gapPinsFA2 = np.zeros(fuelTempsFA2.shape).tolist()

fuelPinsFA3 = np.zeros(fuelTempsFA3.shape).tolist()
# blankPinsFA3 = np.zeros(fuelTempsFA3.shape).tolist()
guidePinsFA3 = np.zeros(fuelTempsFA3.shape).tolist()
gapPinsFA3 = np.zeros(fuelTempsFA3.shape).tolist()

for i in range(0, fuelTempsFA1.shape[0]):
    for j in range(0, fuelTempsFA1.shape[1]):
        fuelPinsFA1[i][j] = pin("fuelPinFA1_Assem"+str(i+1)+"Z"+str(j+1), 3)
        fuelPinsFA1[i][j].set('materials', [fuelMatsFA1[i][j], clad, coolMatsFA1[i][j]])
        fuelPinsFA1[i][j].set('radii', fuelPinRadii)
    
        guidePinsFA1[i][j] = pin("guidePinFA1_Assem"+str(i+1)+"Z"+str(j+1), 3)
        guidePinsFA1[i][j].set('materials', [coolMatsFA1[i][j], clad, coolMatsFA1[i][j]])
        guidePinsFA1[i][j].set('radii', guideTubeRadii)
    
        gapPinsFA1[i][j] = pin("gapPinFA1_Assem"+str(i+1)+"Z"+str(j+1), 3)
        gapPinsFA1[i][j].set('materials', [coolMatsFA1[i][j]])
        
for i in range(0, fuelTempsFA2.shape[0]):
    for j in range(0, fuelTempsFA2.shape[1]):
        fuelPinsFA2[i][j] = pin("fuelPinFA2_Assem"+str(i+1)+"Z"+str(j+1), 3)
        fuelPinsFA2[i][j].set('materials', [fuelMatsFA2[i][j], clad, coolMatsFA2[i][j]])
        fuelPinsFA2[i][j].set('radii', fuelPinRadii)
        
        guidePinsFA2[i][j] = pin("guidePinFA2_Assem"+str(i+1)+"Z"+str(j+1), 3)
        guidePinsFA2[i][j].set('materials', [coolMatsFA2[i][j], clad, coolMatsFA2[i][j]])
        guidePinsFA2[i][j].set('radii', guideTubeRadii)
    
        gapPinsFA2[i][j] = pin("gapPinFA2_Assem"+str(i+1)+"Z"+str(j+1), 3)
        gapPinsFA2[i][j].set('materials', [coolMatsFA2[i][j]])
        

        
for i in range(0, fuelTempsFA3.shape[0]):
    for j in range(0, fuelTempsFA3.shape[1]):
        fuelPinsFA3[i][j] = pin("fuelPinFA3_Assem"+str(i+1)+"Z"+str(j+1), 3)
        fuelPinsFA3[i][j].set('materials', [fuelMatsFA3[i][j], clad, coolMatsFA3[i][j]])
        fuelPinsFA3[i][j].set('radii', fuelPinRadii)
            
        guidePinsFA3[i][j] = pin("guidePinFA3_Assem"+str(i+1)+"Z"+str(j+1), 3)
        guidePinsFA3[i][j].set('materials', [coolMatsFA3[i][j], clad, coolMatsFA3[i][j]])
        guidePinsFA3[i][j].set('radii', guideTubeRadii)
    
        gapPinsFA3[i][j] = pin("gapPinFA3_Assem"+str(i+1)+"Z"+str(j+1), 3)
        gapPinsFA3[i][j].set('materials', [coolMatsFA3[i][j]])

## Generating Channels
1. To model an individual channel we will be using the serpent pinStack lattice
2. Each indivial channel will consist of indivial pins (axial layers)
3. There will be 3 types of channels fuel channels, guide tubes, and water gaps

In [7]:
axialheights = np.array([
    0.000000, 
    20.00000,
    25.00000,
    25.00000 + 10.2777777778*1, 
    25.00000 + 10.2777777778*2,
    25.00000 + 10.2777777778*3,
    25.00000 + 10.2777777778*4,
    25.00000 + 10.2777777778*5,
    25.00000 + 10.2777777778*6,
    25.00000 + 10.2777777778*7,
    25.00000 + 10.2777777778*8,
    25.00000 + 10.2777777778*9,
    25.00000 + 10.2777777778*10,
    25.00000 + 10.2777777778*11,
    25.00000 + 10.2777777778*12,
    25.00000 + 10.2777777778*13,
    25.00000 + 10.2777777778*14,
    25.00000 + 10.2777777778*15,
    25.00000 + 10.2777777778*16,
    25.00000 + 10.2777777778*17,
    210.0000,
    220.0000
])
# print(axialheights)

for i in range(0, fuelTempsFA1.shape[0]):
    fuelPinsFA1[i].insert(0, RB)
    fuelPinsFA1[i].append(RT)
    guidePinsFA1[i].insert(0, RB)
    guidePinsFA1[i].append(RT)
    gapPinsFA1[i].insert(0, RB)
    gapPinsFA1[i].append(RT)
    
    
for i in range(0, fuelTempsFA2.shape[0]):
    fuelPinsFA2[i].insert(0, RB)
    fuelPinsFA2[i].append(RT)
    guidePinsFA2[i].insert(0, RB)
    guidePinsFA2[i].append(RT)
    gapPinsFA2[i].insert(0, RB) 
    gapPinsFA2[i].append(RT)
    
    
for i in range(0, fuelTempsFA3.shape[0]):
    fuelPinsFA3[i].insert(0, RB)   
    fuelPinsFA3[i].append(RT)
    guidePinsFA3[i].insert(0, RB)
    guidePinsFA3[i].append(RT)
    gapPinsFA3[i].insert(0, RB)
    gapPinsFA3[i].append(RT)
    
    
# pinS = pins()
# pinS.addPins(fuelPinsFA1[i])
# print(pinS.toString())

FP1 = np.zeros(nFA1).tolist()
FP2 = np.zeros(nFA2).tolist()
FP3 = np.zeros(nFA3).tolist()

TP1 = np.zeros(nFA1).tolist()
TP2 = np.zeros(nFA2).tolist()
TP3 = np.zeros(nFA3).tolist()

CT1 = np.zeros(nFA1).tolist()
CT2 = np.zeros(nFA2).tolist()
CT3 = np.zeros(nFA3).tolist()

WT1 = np.zeros(nFA1).tolist()
WT2 = np.zeros(nFA2).tolist()
WT3 = np.zeros(nFA3).tolist()


for i in range(0, len(FP1)):
    FP1[i] = pinStack("FC_FA1_Assem"+str(i+1), 0, 0, 22)
    FP1[i].setStack(np.array(fuelPinsFA1[i]), axialheights)
    TP1[i] = pinStack("TC_FA1_Assem"+str(i+1), 0, 0, 22)
    TP1[i].setStack(np.array(guidePinsFA1[i]), axialheights)
    CT1[i] = pinStack("CC_FA1_Assem"+str(i+1), 0, 0, 22)
    CT1[i].setStack(np.array(guidePinsFA1[i]), axialheights)
    WT1[i] = pinStack("WC_FA1_Assem"+str(i+1), 0, 0, 22)
    WT1[i].setStack(np.array(gapPinsFA1[i]), axialheights)
    
for i in range(0, len(FP2)):
    FP2[i] = pinStack("FC_FA2_Assem"+str(i+1), 0, 0, 22)
    FP2[i].setStack(np.array(fuelPinsFA2[i]), axialheights)
    TP2[i] = pinStack("TC_FA2_Assem"+str(i+1), 0, 0, 22)
    TP2[i].setStack(np.array(guidePinsFA2[i]), axialheights)
    CT2[i] = pinStack("CC_FA2_Assem"+str(i+1), 0, 0, 22)
    CT2[i].setStack(np.array(guidePinsFA2[i]), axialheights)
    WT2[i] = pinStack("WC_FA2_Assem"+str(i+1), 0, 0, 22)
    WT2[i].setStack(np.array(gapPinsFA2[i]), axialheights)
    
for i in range(0, len(FP3)):
    FP3[i] = pinStack("FC_FA3_Assem"+str(i+1), 0, 0, 22)
    FP3[i].setStack(np.array(fuelPinsFA3[i]), axialheights)
    TP3[i] = pinStack("TC_FA3_Assem"+str(i+1), 0, 0, 22)
    TP3[i].setStack(np.array(guidePinsFA3[i]), axialheights)
    CT3[i] = pinStack("CC_FA3_Assem"+str(i+1), 0, 0, 22)
    CT3[i].setStack(np.array(guidePinsFA3[i]), axialheights)
    WT3[i] = pinStack("WC_FA3_Assem"+str(i+1), 0, 0, 22)
    WT3[i].setStack(np.array(gapPinsFA3[i]), axialheights)

## Generating Assembly Layout
1. To model the fuel assembly we will be using a sqaure lattice (19 x 19).
2. The assembly will consist of our generated fuel, guide tube, and water gap channels.

In [8]:
FA1 = np.zeros(nFA1).tolist()
FA2 = np.zeros(nFA2).tolist()
FA3 = np.zeros(nFA3).tolist()

for i in range(0, nFA1):
    FA1[i] = sqLat("FA1_"+str(i+1), 0, 0, 19, 1.260)

    FA1Map = np.array([
        [WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], CT1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], TP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], FP1[i], WT1[i]],
        [WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i], WT1[i]],
    ])

    FA1[i].setMap(FA1Map)
    
for i in range(0, nFA2):
    FA2[i] = sqLat("FA2_"+str(i+1), 0, 0, 19, 1.260)

    FA2Map = np.array([
        [WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], CT2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], TP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], FP2[i], WT2[i]],
        [WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i], WT2[i]],
    ])

    FA2[i].setMap(FA2Map)

for i in range(0, nFA3):
    FA3[i] = sqLat("FA3_"+str(i+1), 0, 0, 19, 1.260)

    FA3Map = np.array([
        [WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], CT3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], TP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], FP3[i], WT3[i]],
        [WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i], WT3[i]],
    ])

    FA3[i].setMap(FA3Map)

In [9]:
W1 = pin("W1", 3)
W1.set('materials', [mod1])

WG1 = pinStack("WG1", 0, 0, 3)
WG1.setStack(np.array([RB, W1, RT]), np.array([0.0, 20.0, 220.0]))

WA1 = FA1[0].duplicate("WA1")
WA1.replacePin(FP1[0], WG1)
WA1.replacePin(TP1[0], WG1)
WA1.replacePin(CT1[0], WG1)
WA1.replacePin(WT1[0], WG1)

WA1 = [WA1]

In [10]:
CORE = sqLat("CORE", 0, 0, 11, 21.6038)

COREMap = np.array([
[WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0]],
[WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0]],
[WA1[0], WA1[0], WA1[0], WA1[0], FA3[0], FA3[1], FA3[2], WA1[0], WA1[0], WA1[0], WA1[0]],
[WA1[0], WA1[0], WA1[0], FA3[3], FA2[0], FA2[1], FA2[2], FA3[4], WA1[0], WA1[0], WA1[0]],
[WA1[0], WA1[0], FA3[5], FA2[3], FA1[0], FA1[1], FA1[2], FA2[4], FA3[6], WA1[0], WA1[0]],
[WA1[0], WA1[0], FA3[7], FA2[5], FA1[3], FA1[4], FA1[5], FA2[6], FA3[8], WA1[0], WA1[0]],
[WA1[0], WA1[0], FA3[9], FA2[7], FA1[6], FA1[7], FA1[8], FA2[8], FA3[10], WA1[0], WA1[0]],
[WA1[0], WA1[0], WA1[0], FA3[11], FA2[9], FA2[10], FA2[11], FA3[12], WA1[0], WA1[0], WA1[0]],
[WA1[0], WA1[0], WA1[0], WA1[0], FA3[13], FA3[14], FA3[15], WA1[0], WA1[0], WA1[0], WA1[0]],
[WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0]],
[WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0], WA1[0]]
])

CORE.setMap(COREMap)


# print(CORE.toString())
# text_file = open("sample.txt", "w")
# n = text_file.write(CORE.toString())
# text_file.close()
    

## Generating Core Housing
1. Housing can be either just the closing surface ("border") for the core or can include sheilding, baffle, vessel, barrel, etc.
2. The option for having concentric rings around the assembly housing is also an option.
2. For this scenario the core will be enclosed by concentric rings.

In [11]:
width = 21.6038*8
height  = 240
radii = [0, 20, 40, 60]
mats = [clad, mod1, clad]

h1 = housing(width=width, height=height, defaultCRFlag=True, radiiCR= radii, matsCR= mats)

print(h1.toString())

cell cc1 housing  clad cr1 -cr2 
cell cc2 housing  modFA1 cr2 -cr3 
cell cc3 housing  clad cr3 -cr4 
cell core housing  outside -cr1 

surf cr1 cyl 0.0 0.0 86.4152 
surf cr2 cyl 0.0 0.0 106.4152 
surf cr3 cyl 0.0 0.0 126.4152 
surf cr4 cyl 0.0 0.0 146.4152 

mat clad    -6.55    rgb 183 183 183
40090.06c	0.5145
40091.06c	0.1122
40092.06c	0.1715
40094.06c	0.1738
40096.06c	0.028

mat modFA1    -0.7 moder H2O 1001   rgb 198 226 255
1001.06c	0.66667
8016.06c	0.33333




## Putting it all together
1. The core/assembly layout (main universe) must be given.
2. The housing of the mainuniverse must be given
3. The optional flags parameter can be set `for additional settings

In [12]:
input = core(mainUniv=CORE, housing=h1)

In [13]:
ngroups = 2
ebounds = np.array([0.625E-6])

xsUnivs = [RT, RB]

input.setXS(ngroups=ngroups, ebounds=ebounds, universes= xsUnivs, setFPPXS=True, setADF=True)
print(input.xs['toString'])


set gcu RT  RB  
set nfg 2 6.25e-07 
set poi 1 
set adf 0 voidBorder full




In [14]:
absPath = "/mnt/c/Users/iaguirre6/Documents/xsdata/endfb7"
thermLib = "therm lwtr lwj3.11t"
input.setXSLib(absPath, thermLib)

In [15]:
input.writeFile("test.txt")
text_file = open("test.txt") #Display contents of created inputFile
file_content = text_file.read()
print(file_content)
text_file.close()

lat CORE 1 0 0  11 11 21.6038
WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 FA3_1 FA3_2 FA3_3 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 FA3_4 FA2_1 FA2_2 FA2_3 FA3_5 WA1 WA1 WA1 
WA1 WA1 FA3_6 FA2_4 FA1_1 FA1_2 FA1_3 FA2_5 FA3_7 WA1 WA1 
WA1 WA1 FA3_8 FA2_6 FA1_4 FA1_5 FA1_6 FA2_7 FA3_9 WA1 WA1 
WA1 WA1 FA3_10 FA2_8 FA1_7 FA1_8 FA1_9 FA2_9 FA3_11 WA1 WA1 
WA1 WA1 WA1 FA3_12 FA2_10 FA2_11 FA2_12 FA3_13 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 FA3_14 FA3_15 FA3_16 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 
WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 WA1 

pin RB
reflector

pin W1
modFA1

pin RT
reflector

lat WG1 9 0 0 3
0.0	RB
20.0	W1
220.0	RT

lat WA1 1 0 0  19 19 1.26
WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 
WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 
WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 
WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 WG1 W

###  Core Object Info

In [16]:
vars(input)

{'flagXS': True,
 'flagBurn': False,
 'flagBranch': False,
 'flagSettings': False,
 'input': {'mainLat': <serpentGenerator.functions.sqLattice.sqLat at 0x1ddd1cb5588>,
  'housing': <serpentGenerator.functions.housing.housing at 0x1ddc0c5ddc8>,
  'main': <serpentGenerator.functions.universe.universe at 0x1ddc0c5d988>},
 'materials': None,
 'pins': None,
 'lats': None,
 'mainLat': <serpentGenerator.functions.sqLattice.sqLat at 0x1ddd1cb5588>,
 'housing': <serpentGenerator.functions.housing.housing at 0x1ddc0c5ddc8>,
 'mainUniv': <serpentGenerator.functions.universe.universe at 0x1ddc0c5d988>,
 'voidSurf': <serpentGenerator.functions.surf.surf at 0x1ddc0c5dbc8>,
 'burnup': {},
 'xs': {'ngroups': 2,
  'ebounds': array([6.25e-07]),
  'universes': [<serpentGenerator.functions.pin.pin at 0x1ddc0c338c8>,
  'setFPPXS': True,
  'setADF': True,
  'toString': 'set gcu RT  RB  \nset nfg 2 6.25e-07 \nset poi 1 \nset adf 0 voidBorder full\n\n'},
 'branch': {},
 'settings': {},
 'plot': {},
 'coef': {